In [175]:
import pandas as pd
import numpy as np
import qiime2 as q2
import biom
import skbio

In [247]:
# load metadata 
md = pd.read_csv('../data/subsetted_md.tsv', sep='\t', index_col=0)
md = md.loc[md.index.intersection(md.index)]
# of the categorical columns, remove those that do not have at least 2 categories with less than 5 people in them 
cat_columns = md.dtypes[(md.dtypes != 'float64')].index
cat_to_remove = []
for c in cat_columns: 
    if len(md[c].dropna().unique()) < 2: 
        cat_to_remove.append(c)
    elif md[c].value_counts().min() < 5: 
        cat_to_remove.append(c) 

# also remove columns that are only present in one thdmi cohort
only_one_thdmi_cohort = []
for c in md.columns: 
    sub_md = md.loc[md[c].notna()] 
    if len(sub_md.thdmi_cohort.unique()) == 1: 
        only_one_thdmi_cohort.append(c) 

md.drop(columns = cat_to_remove + only_one_thdmi_cohort, inplace=True)    

# rename columns
md.columns = [c.replace('+', 'and').replace('=', 'to').replace('[', '_').replace(']', '_').replace(';', '_').replace('&', 'and').replace('/', 'or').replace(' ', '') for c in md.columns]

/tmp/ipykernel_222108/2357188733.py:2: DtypeWarning: Columns (45,48,64,70,71,72,73,113,119,140,142,160,165,167,177,195,198,200,201,224,252,261,265,266,284,285,300,304,305,306,315,316,319,321,325,328,329,341,342,347,351,354,358,361,368,369,370,374,376,380,384,389,390,397,399,407,408,416,428,431,438,443,446,450,455,460,461,462,463,464,465,470,475,481,485,487,493,504,543,549,562,573,582,585,591,598,602,607,611,612,613,637,643,667,687,697,698,707,708,709,720,727,737,739,743,748,760,769,773,806,809,813,840,844,850,852,870,874,891,892,899,910,912,924,925,926,933,936,940,946,953,955,956) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('../data/subsetted_md.tsv', sep='\t', index_col=0)


In [248]:
md = md.dropna(axis=1, how='all')

In [249]:
md.rename_axis(index='SampleID', inplace=True)
md.to_csv('../data/md_for_adonis.tsv', sep='\t')

In [250]:
hei = []

for col in md.columns:
    if 'hei2' in col.lower():
        hei.append(col)
        
md = md.drop(columns=hei)

In [251]:
diet_cols = []

diet_names = ['Servings', 'Amount', 'Collapsed', 'MPED', 'Calories', 'Grams']

for name in diet_names:
    for col in md.columns:
        if name.lower() in col.lower():
            diet_cols.append(col)
md = md.drop(columns=diet_cols)

In [252]:
columns_to_examine = md.columns

In [253]:
pd.DataFrame(data={'columns': columns_to_examine}).to_csv("cols_for_examination.csv")

In [199]:
#run 1_run_alpha_beta.sh and 2_run_adonis.sh